In [237]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time

import os #MAKDIR & DELETING FILES
import csv #COMMA SEPARATED VALUES LIBRARY

In [238]:
#TO WŁĄCZA STRONĘ
PATH = r"C:\Users\PC\1st\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/mazowieckie")

In [239]:
def create_folder_in_current_directory(folderName):#FOLDER NAME TO STRING - MA BYĆ W CUDZYSŁOWIE
    try:
        os.mkdir(folderName)
    except:
        print("Nie stworzono folderu o nazwie: '" + folderName+"'")
        
#DIRECTORY MA BYĆ Z UKOŚNIKAMI "/", A NIE "\"!!! POZA TYM OBA ARGUMENTY W CUDZYSŁOWACH
def create_folder_in_given_directory(directory, folderName):
    path = os.path.join(directory, folderName)
    try:
        os.mkdir(path)
    except:
        print("Nie stworzono folderu o nazwie: '" + folderName+"' w: " + path)
        
def append_list_of_links_of_images(list, object):#object holds img elements within itself
    images = object.find_elements_by_class_name("image-gallery-image")
    for i in range(len(images)):
        if(images[i].get_attribute("src") in imagesLinks):
            continue
        else:
            imagesLinks.append(images[i].get_attribute("src"))

In [240]:
#TO TWOŻY LISTĘ LINKÓW "links".
posts = driver.find_elements_by_css_selector("a[data-cy='listing-item-link']")
links = []
#print("To jest lista ogłoszeń:")
for post in posts:
    links.append(post.get_attribute("href"))
    #print(links[posts.index(post)])#To jest tylko do wypisywania linków. Można usunąć.
print("Liczba ogłoszeń: " + str(len(links)))

Liczba ogłoszeń: 27


In [241]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!DODAĆ ITERACJĘ NA KAŻDY LINK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
create_folder_in_given_directory("./scraped", "0")#ZROBIĆ ITERACJĘ Z TYM ZEREM
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!DODAĆ ITERACJĘ NA KAŻDY LINK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
directory = "./scraped/" + "0" + "/"#ZAMIAST ZERA TUTAJ DAĆ TEN ITERATOR STR(I)

Nie stworzono folderu o nazwie: '0' w: ./scraped\0


In [242]:
data = []#TU BĘDĄ WSZYSTKIE PARAMETRY MIESZKANIA
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!DODAĆ ITERACJĘ NA KAŻDY LINK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
driver.get(links[0])#WEJŚĆ NA STRONĘ

In [243]:
cena = driver.find_element_by_css_selector("strong[aria-label='Cena']")
cenaMetra = driver.find_element_by_css_selector("div[aria-label='Cena za metr kwadratowy']")
title = driver.find_element_by_tag_name("h1")
data.append("Tytuł:")
data.append(title.text)
data.append("Cena:")
data.append(cena.text)
data.append("Cena za metr kwadratowy:")
data.append(cenaMetra.text[:-1])

In [244]:
table = driver.find_element_by_class_name("egzohkh2")#ZNAJDUJE TABELEKĘ ZE SZCZEGÓŁAMI
fields = table.find_elements_by_xpath(".//*")#ZNAJDUJE WSZYSTKIE ELEMENTY W TABELCE
for field in fields:
    teksty = field.find_elements_by_xpath(".//*");#ZEBRAĆ WSZYSTKIE JEJ ELEMENTY (TYTUŁ I WARTOŚĆ, ZDJĘCIE Z JAKIEGOŚ POWODU SIE NIE ZNAJDUJE)
    if (fields.index(field)!=0):#METRY KWADRATOWE WYWALAŁY BŁĄD, WIĘC POWIERZCHNIA JEST OSOBNO
        if(len(teksty)==2):
            data.append(teksty[0].text)
            data.append(teksty[1].text)
    else:#TO JEST POWIERZCHNIA
        if(len(teksty)==2):
            data.append(teksty[0].text)#WPISUJE WARTOŚCI DO TABLICY, Z KTÓREJ PÓŹNIEJ BĘDZIE WPISYWANE DO PLIKU
            temp = teksty[1].text
            temp = temp[:-1]
            data.append(temp)
print (data)

['Tytuł:', '75m2 + ogródek 225m2 ul. Celulozy KLUCZE 12.2021', 'Cena:', '719 000 zł', 'Cena za metr kwadratowy:', '9 587 zł/m', 'Powierzchnia:', '75 m', 'Liczba pokoi:', '4', 'Rynek:', 'pierwotny', 'Rodzaj zabudowy:', 'dom wolnostojący', 'Liczba pięter:', '1', 'Materiał budynku:', 'cegła', 'Okna:', 'plastikowe', 'Rok budowy:', '2021', 'Stan wykończenia:', 'do wykończenia', 'Forma własności:', 'pełna własność']


In [245]:
with open(directory+'dane.csv', 'w', newline='') as csvfile:#OTWIERA DANE.CSV I USUWA WSZYSTKO CO W NICH BYŁO JEŻELI COŚ BYŁO
    pisacz = csv.writer(csvfile, delimiter=';')#POTRZEBNE DLA BIBLIOTEKI CSV
    j = 0
    i = 0
    if (len(data) > 2):
        for i in range(0, (int(len(data)/2))):
            try:
                pisacz.writerow([data[j],data[j+1]])#PISZE PO DWIE KOMÓRKI Z DANYCH NA WIERSZ
                j+=2
            except:
                print("Nie udało się coś")
    else:
        print("Za mało danych")

In [246]:
#NIE TYLKO DODATKOWE INFORMACJE MAJĄ TĄ SAMĄ DLASĘ, ALE JAKIEŚ RZECZY TYPU WYPOSARZENIE, ALBO ZABEZPIECZENIA
infDodatkowe = driver.find_elements_by_class_name("ex3yvbv1")#LOKALIZUJE DODATKOWE INFORMACJE
listaTekstow = ["Informacje dodatkowe:"]
link = ["Link:"]
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!DODAĆ ITERACJĘ NA KAŻDY LINK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
link.append(links[0])
for i in range(len(infDodatkowe)):
    listaTekstow.append(infDodatkowe[i].text)
with open(directory+'dane.csv', 'a', newline='') as csvfile:
    pisacz = csv.writer(csvfile, delimiter=';')
    pisacz.writerow(listaTekstow)

In [247]:
opis = driver.find_element_by_css_selector('div[data-cy="adPageAdDescription"]')#ZNAJDUJE OPIS, PO JEGO DATA-CY
teksty = opis.find_elements_by_tag_name("p");#ZNAJDUJE WSZYSTKIE DZIECI OPISU
txtf = open(directory+"desription.txt", "w")#TWORZY PLIK TEKSTOWY
for i in range(len(teksty)):
    txtf.write(teksty[i].text + "\n")#ZAPISUJE WSZYSTKIE LINIJKI OPISU PO KOLEI 
txtf.close()#ZAMYKA PLIK

In [251]:
guzikNext = driver.find_element_by_css_selector("button[data-cy='ad-media-gallery.next-photo']")
try:  
    guzikCookies = driver.find_element_by_id("onetrust-accept-btn-handler")
    guzikCookies.click()
except:
    print("Nie pominięto
liczbaZdjec = int(driver.find_element_by_class_name("image-gallery-index-total").text)
zdjecie = driver.find_element_by_class_name("image-gallery-slides")
imagesLinks = []
for i in range(liczbaZdjec):
    guzikNext.click()
    time.sleep(1)
    append_list_of_links_of_images(imagesLinks, zdjecie)
print (imagesLinks)


IndentationError: expected an indented block (Temp/ipykernel_13484/3289228662.py, line 7)

In [218]:
print(len(imagesLinks))
create_folder_in_given_directory(directory, "zdjecia")
for i in renage(len(imagesLinks)):
    nazwaPliku = "photo" + str(i)+".png"
    path = os.path.join(directory, "zdjecia", nazwaPliku)
    with open(path, 'wb') as file:
        zdjecie = driver.get_element_by_tag_name("img")
        file.write(zdjecie.screenshot_as_png)

7


In [221]:
nazwaPliku = "photo" + str(i)+".png"
create_folder_in_given_directory(directory, "zdjecia")
path = os.path.join(directory, "zdjecia", nazwaPliku)
print(path)

./scraped/0/zdjecia\photo6.png
